In [ ]:
#!/usr/bin/env python3
"""
USDA Nutrition Data Importer

Imports USDA nutrition data from CSV file into MySQL database.
Handles data cleaning and NULL value conversion for numeric fields.
"""

import csv
import mysql.connector
from mysql.connector import Error
import re

# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'user': 'username',
    'password': 'ABCD1234',
    'database': 'usda',
    'auth_plugin': 'mysql_native_password'
}

TABLE_NAME = 'nutrition'
CSV_FILE = 'C:\\Users\\ritvi\\OneDrive\\Desktop\\learnwithvik\\01 - Projects (P)\\01_CSVtoSQL - Nutritional Facts Database\\USDA.csv'

def converter_float(value):
    """
    Convert string values to float, handling edge cases and NULL values.
    
    Args:
        value: Input value to convert
        
    Returns:
        float or None: Converted numeric value or None for invalid/empty data
    """
    try:
        # Clean the input value
        cleaned = str(value).replace(',', '').strip()
        
        # Remove non-numeric characters except decimal point, minus, and scientific notation
        cleaned = re.sub(r'[^\d.\-eE+]', '', cleaned)
        
        # Handle empty or placeholder values
        if cleaned == '' or cleaned.lower() in ['n/a', 'na', '--', 'null']:
            return None
            
        return float(cleaned)
    except Exception:
        return None


def import_csv():
    """
    Main import function that connects to MySQL and imports CSV data.
    Creates table schema and handles data insertion with proper error handling.
    """
    connection = None
    try:
        # Connect to MySQL database
        print("Connecting to MySQL database...")
        connection = mysql.connector.connect(**DB_CONFIG)
        
        if connection.is_connected():
            cursor = connection.cursor()
            print(f"✓ Connected to MySQL {connection.get_server_info()}")
            
            # Create table with proper schema
            print("\nCreating table...")
            cursor.execute(f"DROP TABLE IF EXISTS {TABLE_NAME}")
            
            create_sql = """
            CREATE TABLE nutrition (
                id INT AUTO_INCREMENT PRIMARY KEY,
                description VARCHAR(255),
                calories FLOAT,
                protein FLOAT,
                total_fat FLOAT,
                carbohydrate FLOAT,
                sugar FLOAT
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
            """
            cursor.execute(create_sql)
            connection.commit()
            print("✓ Table created successfully")

            # Import CSV data
            print("\nImporting data...")
            with open(CSV_FILE, 'r', encoding='utf-8') as file:
                csv_reader = csv.DictReader(file)
                
                for row in csv_reader:
                    # Debug output for first row
                    if csv_reader.line_num == 2:
                        print("Sample data row:", row)
                    
                    # Insert row with proper data conversion
                    cursor.execute("""
                        INSERT INTO nutrition 
                        (description, calories, protein, total_fat, carbohydrate, sugar)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (
                        row['Description'],
                        converter_float(row['Calories']),
                        converter_float(row['Protein']),
                        converter_float(row['TotalFat']),
                        converter_float(row['Carbohydrate']),
                        converter_float(row['Sugar'])
                    ))
                    
                    # Progress indicator
                    if csv_reader.line_num % 100 == 0:
                        print(f"Imported {csv_reader.line_num - 1} rows...")
            
            connection.commit()
            print(f"\n✓ Successfully imported {csv_reader.line_num - 1} rows")

            # Verify import with sample data
            cursor.execute("SELECT * FROM nutrition LIMIT 5")
            print("\nSample imported data:")
            for row in cursor.fetchall():
                print(row)
                
    except Error as e:
        print(f"\nMySQL Error: {e}")
    except Exception as e:
        print(f"\nError: {e}")
    finally:
        if connection and connection.is_connected():
            connection.close()
            print("\nMySQL connection closed")


if __name__ == "__main__":
    print("=== USDA Nutrition Data Importer ===")
    import_csv()

=== USDA Nutrition Data Importer ===
Connecting to MySQL database...
✓ Connected to MySQL 8.0.42

Creating table...
✓ Table created successfully

Importing data...
Sample data row: {'ID': '14034', 'Description': 'ALCOHOLIC BEV,CREME DE MENTHE,72 PROOF', 'Calories': '371', 'Protein': ' -   ', 'TotalFat': '0.3', 'Carbohydrate': '41.6', 'Sodium': '5', 'SaturatedFat': '0.01', 'Cholesterol': ' -   ', 'Sugar': '41.6', 'Calcium': ' -   ', 'Iron': '0.07', 'Potassium': ' -   ', 'VitaminC': ' -   ', 'VitaminE': ' -   ', 'VitaminD': ' -   '}
Imported 99 rows...
Imported 199 rows...
Imported 299 rows...
Imported 399 rows...
Imported 499 rows...
Imported 599 rows...
Imported 699 rows...
Imported 799 rows...
Imported 899 rows...
Imported 999 rows...
Imported 1099 rows...
Imported 1199 rows...
Imported 1299 rows...
Imported 1399 rows...
Imported 1499 rows...
Imported 1599 rows...
Imported 1699 rows...
Imported 1799 rows...
Imported 1899 rows...
Imported 1999 rows...
Imported 2099 rows...
Imported 219